In [1]:
import face_recognition
import cv2
import os
import pickle
import numpy as np
import time
import pandas as pd
from csv import writer

In [2]:
folderpath = 'Faces'
filepath = os.listdir(folderpath)

img_list = []
studentIds = []

for path in filepath:
    img_list.append(cv2.imread(os.path.join(folderpath, path)))
    studentIds.append(os.path.splitext(path)[0])

In [3]:
def findEncodings(ImagesList):
    
    encodeList = []
    for img in ImagesList:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)

    return encodeList

In [4]:
print("Start...")
encodeListKnow = findEncodings(img_list)
encodeListKnowWithIds = [encodeListKnow,studentIds]
print("Complete")


Start...
Complete


In [5]:
file = open('EncodingFile.p','wb')
pickle.dump(encodeListKnowWithIds,file)
file.close()

In [6]:
file = open('EncodingFile.p','rb')
encodeListKnowWithIds = pickle.load(file)
file.close()

In [7]:
encodeListKnow , studentIds = encodeListKnowWithIds

In [8]:
df = pd.read_csv('Attendance.csv')
index_csv = df['Index']
name_csv = df['Name']
date_csv = df['Date']
time_csv = df['Time']

In [9]:
while True:
    cap = cv2.VideoCapture(0)
    ret,frame = cap.read()

    imgS = cv2.resize(frame,(0,0),None,0.5,0.5)
    imgS = cv2.cvtColor(imgS,cv2.COLOR_BGR2RGB)


    faceCurFrame = face_recognition.face_locations(imgS)
    encodeCurFrame = face_recognition.face_encodings(imgS,faceCurFrame)

    for encodeFace,faceLoc in zip(encodeCurFrame,faceCurFrame):
        matches = face_recognition.compare_faces(encodeListKnow,encodeFace)
        faceDis = face_recognition.face_distance(encodeListKnow,encodeFace)
        matcheIndex = np.argmin(faceDis)

    
        if matches[matcheIndex]:
            index_csv = matcheIndex
            name_csv = studentIds[matcheIndex]
            date_csv = time.strftime("%d/%m/%Y")
            time_csv = time.strftime("%H:%M:%S")
            list = [index_csv,name_csv,date_csv,time_csv]
            
            with open('Attendance.csv', 'a') as f_object:
                writer_object = writer(f_object)
                writer_object.writerow(list)
                f_object.close()


    cv2.imshow("face-reconition",frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()